## 多層ニューラルモデルクラスの定義
多層パーセプトロン (Multi Layer Perceptron, MLP) とも呼ばれる

In [25]:
import torch
from torch import nn
from torch import optim

class MultiLayer(nn.Module):
    """
    多層モデルクラス
    """
    def __init__(self):
        super(MultiLayer, self).__init__()
        self.linear1 = torch.nn.Linear(1, 1) # wx+b 初期パラメータはランダム
        self.linear2 = torch.nn.Linear(1, 1)
        self.linear3 = torch.nn.Linear(1, 1)

        # 損失関数
        self.loss_func = nn.MSELoss()

        
    def forward(self, input_vector):
        """
        順伝搬
        :param input_vector: 入力ベクトル
        :return tanh_w3x: モデルの出力
        """
        w1x = self.linear1(input_vector)
        tanh_w1x = torch.tanh(w1x) # torch.nn.functional.tanhはdeprecated
        w2x = self.linear2(tanh_w1x)
        tanh_w2x = torch.tanh(w2x) 
        w3x = self.linear3(tanh_w2x)
        tanh_w3x = torch.tanh(w3x)
        return tanh_w3x

    
    def forward_loss(self, input_vector, target_values):
        """
        順伝搬 + 損失計算
        :param input_vector: 入力ベクトル
        :param target_values: 正解の値
        :return loss:損失
        """
        wx = self.forward(input_vector)
        loss = self.loss_func(wx, target_values)
        return loss

In [26]:
# モデルのインスタンス
multi_layer_model = MultiLayer()

## 最急勾配法による最適化

In [27]:
op = optim.SGD(multi_layer_model.parameters(), lr=0.1) # lr:learning rate (学習率)

## 順伝搬

In [28]:
# 入力と正解
input_vector = torch.rand(5, 1)
target_values = torch.reshape(torch.tensor([[i[0]+1 for i in input_vector]]), (5, 1))

In [29]:
# 順伝搬と損失計算
loss = multi_layer_model.forward_loss(input_vector, target_values)
print(loss)

tensor(4.5501, grad_fn=<MseLossBackward>)


## 逆伝搬

In [30]:
# 逆伝搬 (勾配の設定)
loss.backward()

In [35]:
# 勾配の確認
print(multi_layer_model.linear1.weight.grad)
print(multi_layer_model.linear1.bias.grad)
print(multi_layer_model.linear2.weight.grad)
print(multi_layer_model.linear2.bias.grad)
print(multi_layer_model.linear3.weight.grad)
print(multi_layer_model.linear3.bias.grad)

tensor([[0.0033]])
tensor([0.0087])
tensor([[0.0277]])
tensor([-0.4345])
tensor([[-0.1025]])
tensor([-1.8307])


## パラメータ更新

In [32]:
# 更新前パラメータの確認
print(multi_layer_model.linear1.weight)
print(multi_layer_model.linear1.bias)
print(multi_layer_model.linear2.weight)
print(multi_layer_model.linear2.bias)
print(multi_layer_model.linear3.weight)
print(multi_layer_model.linear3.bias)

Parameter containing:
tensor([[-0.5300]], requires_grad=True)
Parameter containing:
tensor([0.1402], requires_grad=True)
Parameter containing:
tensor([[-0.0203]], requires_grad=True)
Parameter containing:
tensor([0.0548], requires_grad=True)
Parameter containing:
tensor([[0.2381]], requires_grad=True)
Parameter containing:
tensor([-0.9967], requires_grad=True)


In [33]:
# gradに基づくパラメータ更新
op.step()

In [34]:
# 更新後メータの確認
print(multi_layer_model.linear1.weight)
print(multi_layer_model.linear1.bias)
print(multi_layer_model.linear2.weight)
print(multi_layer_model.linear2.bias)
print(multi_layer_model.linear3.weight)
print(multi_layer_model.linear3.bias)

Parameter containing:
tensor([[-0.5303]], requires_grad=True)
Parameter containing:
tensor([0.1394], requires_grad=True)
Parameter containing:
tensor([[-0.0230]], requires_grad=True)
Parameter containing:
tensor([0.0982], requires_grad=True)
Parameter containing:
tensor([[0.2483]], requires_grad=True)
Parameter containing:
tensor([-0.8136], requires_grad=True)


In [24]:
# 勾配の消去
multi_layer_model.zero_grad()

## 学習：勾配に基づくパラメータ更新の繰り返し
順伝搬 -> 損失計算 -> 逆伝搬 -> 勾配計算 -> パラメータ更新 -> 順伝搬 -> ・・・

In [38]:
for epoch in range(3):
    # 順伝搬と損失計算
    loss = multi_layer_model.forward_loss(input_vector, target_values)

    # 逆伝搬 (勾配の設定)
    loss.backward()
  
    # パラメータの更新
    op.step()

    # 勾配の消去
    multi_layer_model.zero_grad()
